<a href="https://colab.research.google.com/github/KaiaX926/P-Stocks-US/blob/main/SANA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import json
from urllib.request import urlopen
import datetime
import certifi
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import time

%matplotlib inline

YOUR_API_KEY = '2c2cd38706ef0b12bdbf80a33806aa32'
today = datetime.datetime.now().strftime("%Y-%m-%d")
startdate = (datetime.datetime.today() - datetime.timedelta(days = 366*3+31)).strftime("%Y-%m-%d")

- **fed funds rate** is the rate at which the fed suggests commercial banks borrow and lend their excess reserves to each other overnight. When the Fed raises the fed funds rate, it’s aiming to increase short-term interest rates throughout the economy. This in turn reduces the supply of money and makes it more expensive for all kinds of economic participants to borrow money. Lowering the fed funds rate has the opposite effect. It reduces short-term interest rates throughout the economy, increasing the supply of money and making it cheaper to get credit. This may cause moments of low or negative inflation to turn around and may drive hiring as companies are able to grow more cheaply.


- The **Consumer Price Index (CPI)** is a measure of the average change over time in the prices paid by urban consumers for a market basket of consumer goods and services. Indexes are available for the U.S. and various geographic areas. Average price data for select utility, automotive fuel, and food items are also available.

- **Nonfarm payrolls** is the measure of the number of workers in the U.S. excluding farm workers and workers in a handful of other job classifications. 

- **The Industrial Production Index (INDPRO)** is an economic indicator that measures real output for all facilities located in the United States manufacturing, mining, and electric, and gas utilities

In [51]:
from IPython.core.display import update_display
class StockAnalysis():
    def __init__(self):
        self.YOUR_API_KEY = '2c2cd38706ef0b12bdbf80a33806aa32'
        self.today = datetime.datetime.now().strftime("%Y-%m-%d")
        self.startdate = (datetime.datetime.today() - datetime.timedelta(days = 366*3+31)).strftime("%Y-%m-%d")


    #####################################################################
    ## methods
    def get_data(self,url):
        response = urlopen(url)
        data = response.read().decode("utf-8")
        data = json.loads(data)
        return pd.json_normalize(data)

    def read_respond(self,ind,pe,da,response):
        data = response.read().decode("utf-8")
        data = data.replace('\n','')
        data = data.replace('{','')
        data = data.replace('}','')
        data = data[1:-1].split(',')
        #print(data)
        for i in data:
            if '"date" : "' in i:
                i = i.split(':')[1][2:-1]
                da.append(i)
            if '"industry" : "' in i:
                i = i.split(':')[1][2:-1]
                ind.append(i)
            if 'pe" : "' in i:
                p = round(float(i.split(':')[1][2:-2]),3)
                pe.append(p)
        return ind,pe,da

    #####################################################################
    # Analysis

    def get_eco(self):
        indicators = ['CPI','realGDP','durableGoods','totalNonfarmPayroll','industrialProductionTotalIndex','consumerSentiment']
        # cpi = 1, realGDP = /100, durableGoods = /1000, totalNonfarmPayroll = /1000, industrialProductionTotalIndex = 1, consumerSentiment = 1 (higher the more positive)
        indicators_rate = ['federalFunds','unemploymentRate','inflationRate']
        # federalFunds = 1,unemploymentRate = /10,inflationRate  = 1
        
        daily = pd.DataFrame(columns = ['indicator','date','value'])
        daily_rate = pd.DataFrame(columns = ['indicator','date','value'])
            
        for rate in indicators_rate:
            url_daily = f"https://financialmodelingprep.com/api/v4/economic?name={rate}&from={self.startdate}&to={self.today}&apikey={self.YOUR_API_KEY}"
            r = self.get_data(url_daily)
            if rate in ['unemploymentRate']:
                r['value'] = r['value']/10
            r['indicator'] = rate
            daily_rate = pd.concat([daily_rate,r], ignore_index = True)    
        fig_rate = px.line(daily_rate, x="date", y="value", color='indicator')
        fig_rate.show()
        
            
        for indica in indicators:
            url_daily = f"https://financialmodelingprep.com/api/v4/economic?name={indica}&from={self.startdate}&to={self.today}&apikey={self.YOUR_API_KEY}"
            d = self.get_data(url_daily)
            if indica in ['realGDP']:
                d['value'] = d['value']/100
            if indica in ['durableGoods','totalNonfarmPayroll']:
                d['value'] = d['value']/1000
            d['indicator'] = indica
            daily = pd.concat([daily,d], ignore_index = True)
        fig = px.line(daily, x="date", y="value", color='indicator')
        fig.show()
        
        return daily, daily_rate


    def get_overall(self):
        industry_list_NYSE = pd.DataFrame(columns = ['date','industry','pe'])
        industry_list_NASDAQ = pd.DataFrame(columns = ['date','industry','pe'])

        ind_NYSE,pe_NYSE,da_NYSE = [],[],[]
        ind_NASDAQ,pe_NASDAQ,da_NASDAQ = [],[],[]
        
        for day in range(1,365):
            date = (datetime.datetime.today() - datetime.timedelta(days=day)).strftime("%Y-%m-%d")
        
            #industry_list
            url_daily_industry_NYSE = f"https://financialmodelingprep.com/api/v4/industry_price_earning_ratio?date={date}&exchange=NYSE&apikey={self.YOUR_API_KEY}"
            url_daily_industry_NASDAQ = f"https://financialmodelingprep.com/api/v4/industry_price_earning_ratio?date={date}&exchange=NASDAQ&apikey={self.YOUR_API_KEY}"
            
            response_NYSE = urlopen(url_daily_industry_NYSE)
            response_NASDAQ = urlopen(url_daily_industry_NASDAQ)

            ind_NYSE,pe_NYSE,da_NYSE = self.read_respond(ind_NYSE,pe_NYSE,da_NYSE,response_NYSE)
            ind_NASDAQ,pe_NASDAQ,da_NASDAQ = self.read_respond(ind_NASDAQ,pe_NASDAQ,da_NASDAQ,response_NASDAQ)


        print('Overall PE of NYSE:', np.median(pe_NYSE))
        print('Overall PE of NASDAQ:', np.median(pe_NASDAQ))
        upper = (np.median(pe_NYSE) + np.median(pe_NASDAQ)) / 2 * 0.7
        lower = (np.median(pe_NYSE) + np.median(pe_NASDAQ)) / 2 * 0.8
        print('Interval accepted:', lower, upper)
        industry_consider = industry_list_NYSE[industry_list_NYSE]

        current = (datetime.datetime.today() - datetime.timedelta(days = 60)).strftime("%Y-%m-%d")
        pleau = (datetime.datetime.today() - datetime.timedelta(days = 5)).strftime("%Y-%m-%d")

        
        industry_list_NYSE['date'] = da_NYSE
        industry_list_NYSE['industry'] = ind_NYSE
        industry_list_NYSE['pe'] = pe_NYSE
        

        curr_NYSE = np.unique(list(industry_list_NYSE[industry_list_NYSE.date > current]['industry']))
        industry_list_NYSE = industry_list_NYSE[industry_list_NYSE.industry.isin(curr_NYSE)]
        out_NYSE = np.median(industry_list_NYSE[industry_list_NYSE.date > pleau]['pe'])


#        print('Outperforming Industries NYSE:', pe_NYSE)
        fig_NYSE = px.line(industry_list_NYSE, x="date", y="pe", color='industry',title='NYSE')
        fig_NYSE.add_hline(y=out_NYSE, line_width=2, line_dash="dash", line_color="black")      
        fig_NYSE.show()  
        
        print('NYSE:', np.unique(industry_list_NYSE['industry']))


        
        industry_list_NASDAQ['date'] = da_NASDAQ
        industry_list_NASDAQ['industry'] = ind_NASDAQ
        industry_list_NASDAQ['pe'] = pe_NASDAQ

        curr_NASDAQ = np.unique(list(industry_list_NASDAQ[industry_list_NASDAQ.date > current]['industry']))
        industry_list_NASDAQ = industry_list_NASDAQ[industry_list_NASDAQ.industry.isin(curr_NASDAQ)]
        out_NASDAQ = np.median(industry_list_NASDAQ[industry_list_NASDAQ.date > pleau]['pe'])

#        print('Outperforming Industries NASDAQ:', pe_NASDAQ)
        fig_NASDAQ = px.line(industry_list_NASDAQ, x="date", y="pe", color='industry',title='NASDAQ')
        fig_NASDAQ.add_hline(y=out_NASDAQ, line_width=2, line_dash="dash", line_color="black") 
        fig_NASDAQ.show()
        print('NASDAQ:', np.unique(industry_list_NASDAQ['industry']))
        
        return industry_list_NYSE, industry_list_NASDAQ


    def find_industry(self,ind):
        
        url1 = f'https://financialmodelingprep.com/api/v3/quotes/nyse?apikey={self.YOUR_API_KEY}'
        url2 = f'https://financialmodelingprep.com/api/v3/quotes/nasdaq?apikey={self.YOUR_API_KEY}'
        stocks1 = self.get_data(url1)#.columns
        stocks2 = self.get_data(url2)#.columns

        stocks = np.unique(list(stocks1['symbol']) + list(stocks2['symbol']))
        print('Total stocks considered: ',len(stocks))
        data_profile = pd.DataFrame(columns=['symbol','industry', 'price', 'range', 'beta','isEtf', 'isFund'])
        data_score = pd.DataFrame(columns = ['symbol','altmanZScore','piotroskiScore'])
        data_income_statement = pd.DataFrame(columns = ['symbol','reportedCurrency','grossProfitRatio','ebitdaratio','operatingIncomeRatio','netIncomeRatio'])
        data_suggestion = pd.DataFrame(columns=['symbol', 'date', 'rating', 'ratingScore','ratingDetailsDCFScore','ratingDetailsROEScore', 
                                  'ratingDetailsROAScore', 'ratingDetailsDEScore','ratingDetailsPEScore', 'ratingDetailsPBScore'])

        url_sic = f'https://financialmodelingprep.com/api/v4/standard_industrial_classification_list?apikey={YOUR_API_KEY}'
        data_sic = self.get_data(url_sic)
        indslist = list(data_sic['industryTitle'])
        siccode = []
        for i in range(len(indslist)):
            for dd in ind:
                if dd.lower() in indslist[i].lower():
                    siccode.append(data_sic['sicCode'][i])
        print(siccode)        
        url_stocks = f'https://financialmodelingprep.com/api/v4/standard_industrial_classification/all?apikey={YOUR_API_KEY}'
        data_stocks = self.get_data(url_stocks)
        companies_or = list(data_stocks[data_stocks['sicCode'].isin(siccode)]['symbol'])
        print('Stocks in industry:',companies_or)

        for sym in companies_or:
            #print('yes')
            url_profile = f"https://financialmodelingprep.com/api/v3/profile/{sym}?apikey={self.YOUR_API_KEY}"
            x = self.get_data(url_profile)
            if len(x) > 0:
                #print(x['symbol'][0],x['isin'][0])
                if x['currency'][0] in ['USD','AUD','EUR','HKD','CNY'] and x['isin'][0]:
                    x = x[['symbol','industry', 'price', 'range', 'beta','isEtf', 'isFund']]
                    data_profile = pd.concat([data_profile,x], ignore_index = True)
            # else:
            #     companies.remove(sym)

        #####################################################################
        # Personal preference
        data_profile = data_profile[data_profile.price > 20] 

        companies = list(data_profile['symbol'])
        print('companies list is ready')
        print('---'*8)

        data_profile = data_profile.set_index('symbol')
        pricerange = data_profile['range'].str.split('-', expand=True)
        data_profile['currentLoc'] = (data_profile['price'] - pricerange[0].astype('float'))/(pricerange[1].astype('float') - pricerange[0].astype('float')).astype('float64')
        data_profile.currentLoc = data_profile.currentLoc.astype('float')
        data_profile = data_profile.round(2)


        for sym in companies:
            url_score = f"https://financialmodelingprep.com/api/v4/score?symbol={sym}&apikey={self.YOUR_API_KEY}"
            url_income_statement = f"https://financialmodelingprep.com/api/v3/income-statement/{sym}?apikey={self.YOUR_API_KEY}"
            url_suggestion = f"https://financialmodelingprep.com/api/v3/rating/{sym}?apikey={self.YOUR_API_KEY}"
            
            score = self.get_data(url_score)
            if len(score) == 0:
                score = pd.DataFrame([[sym,np.nan,np.nan]], columns = ['symbol','altmanZScore','piotroskiScore'])
            #score['symbol'] = sym
            data_score = pd.concat([data_score,score[['symbol','altmanZScore','piotroskiScore']]], ignore_index = True)
            
            # add preference netIncomeRatio > 0 in the most recent report
            income_statement = self.get_data(url_income_statement)
            if len(income_statement) == 0 or not income_statement['netIncomeRatio'][0] or not income_statement['operatingIncomeRatio'][0] or \
            float(income_statement['netIncomeRatio'][0])< 0 or float(income_statement['operatingIncomeRatio'][0]) < 0 or \
            float(income_statement['operatingIncomeRatio'][0]) < float(income_statement['netIncomeRatio'][0]):
                pass
            else:
                income_statement = pd.DataFrame(income_statement.iloc[0,:]).T
                data_income_statement = pd.concat([data_income_statement, income_statement[['symbol','reportedCurrency','grossProfitRatio','ebitdaratio','operatingIncomeRatio','netIncomeRatio']]],ignore_index = True)
            

            suggestion = self.get_data(url_suggestion)
            if len(suggestion) == 0:
                suggestion = pd.DataFrame([[sym,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,]], 
                        columns=['symbol', 'date', 'rating', 'ratingScore','ratingDetailsDCFScore','ratingDetailsROEScore', 
                                  'ratingDetailsROAScore', 'ratingDetailsDEScore','ratingDetailsPEScore', 'ratingDetailsPBScore'])
            else:               
                suggestion = suggestion[['symbol', 'date', 'rating', 'ratingScore','ratingDetailsDCFScore','ratingDetailsROEScore', 
                                  'ratingDetailsROAScore', 'ratingDetailsDEScore','ratingDetailsPEScore', 'ratingDetailsPBScore']]
            data_suggestion = pd.concat([data_suggestion, suggestion],ignore_index = True)
      
        income_statement_list = list(data_income_statement['symbol'])
        print(f'companies three elements are ready ({len(income_statement_list)} in total):',income_statement_list)
        
        data_score = data_score.set_index('symbol')
        data_income_statement = data_income_statement.set_index('symbol')
        
        
        data_profile = data_profile.join(data_score, how = 'left')
        data_profile = data_profile.join(data_income_statement, how = 'inner')
        
        url_sentiment = f"https://financialmodelingprep.com/api/v4/social-sentiment/trending?apikey={self.YOUR_API_KEY}"
        data_sentiment = self.get_data(url_sentiment)       
        data_sentiment =  data_sentiment[['symbol','stocktwitsSentiment','twitterSentiment']]
        data_sentiment = data_sentiment.set_index('symbol')
        data_profile = data_profile.join(data_sentiment, how = 'left')
        
        url_sentiment_change = f"https://financialmodelingprep.com/api/v4/social-sentiments/change?type=bullish&source=stocktwits&apikey={self.YOUR_API_KEY}"
        data_sentiment_change = self.get_data(url_sentiment_change)       
        data_sentiment_change =  data_sentiment_change[['symbol','sentimentChange','rank']]
        data_sentiment_change.rename(columns={'rank': 'sentimentChangeRank'}, inplace=True)
        data_sentiment_change = data_sentiment_change.set_index('symbol')
        data_profile = data_profile.join(data_sentiment_change, how = 'left')
        
        data_suggestion = data_suggestion[data_suggestion.symbol.isin(income_statement_list)]
        data_suggestion = data_suggestion.set_index('symbol')
        #data_profile = data_profile.fillna(np.nan)
        display(data_profile)
        data_profile = data_profile.astype({'price': 'float64', 'beta': 'float64', 'currentLoc': 'float64', 'altmanZScore': 'float64',
                                           'piotroskiScore': 'float64','grossProfitRatio': 'float64','ebitdaratio': 'float64','operatingIncomeRatio': 'float64',
                                           'netIncomeRatio': 'float64'})
        return data_profile, data_suggestion
            
    def stock(self,compines):
        data_stock = pd.DataFrame(columns = ["symbol","date","open","high","low","close","volume","wma"])
        data_sentiment = pd.DataFrame(columns = ['symbol','date','stocktwitsSentiment','twitterSentiment'])
        
        for sym in compines:
            url_price = f"https://financialmodelingprep.com/api/v3/technical_indicator/daily/{sym}?period=10&type=wma&apikey={self.YOUR_API_KEY}"
            url_sentiment = f"https://financialmodelingprep.com/api/v4/historical/social-sentiment?symbol={sym}&page=0&apikey={self.YOUR_API_KEY}"

            
            stock = self.get_data(url_price)
            stock['symbol'] = sym
            data_stock = pd.concat([data_stock,stock], ignore_index = True)

            fig = go.Figure(data=[go.Candlestick(
                x=stock['date'],
                open=stock['open'],
                high=stock['high'],
                low=stock['low'],
                close=stock['close'])],
                layout_title_text=sym
                )
            fig.update_layout(xaxis_rangeslider_visible=False)
            fig.show()
        
            sentiment = self.get_data(url_sentiment)
            if len(sentiment) > 0:
                sentiment = sentiment[['symbol','date','stocktwitsSentiment','twitterSentiment']]
                data_sentiment = pd.concat([data_sentiment,sentiment], ignore_index = True)
        
        if len(data_sentiment) > 0:
            fig_sentiment = px.line(data_sentiment, x="date", y="stocktwitsSentiment", color='symbol')
            fig_sentiment.show()
        
        return data_stock,data_sentiment

        
    def mockowen(self,compines, datebuy, datesell, invest):
        data_stock = pd.DataFrame(columns = ["symbol","open","high","low","close","wma",'flucRate','earning'])
        
        for sym in compines:
            url_price = f"https://financialmodelingprep.com/api/v3/technical_indicator/daily/{sym}?period=10&type=wma&apikey={self.YOUR_API_KEY}"

            stock = self.get_data(url_price)
            stock = stock.set_index("date")
            stock['symbol'] = sym
            stock['flucRate'] = (stock.open - stock.open.shift(-1))/stock.open.shift(-1)
            stock['earning'] = stock.open - stock.open[datebuy]
            data_stock = pd.concat([data_stock,stock.loc[datesell:datebuy, :]])
            print(f"income on stock {sym} is: {invest/stock['open'][datebuy] * (stock['open'][datesell] - stock['open'][datebuy]):0.4f}")
        
            fig_stock = px.line(stock.loc[datesell:datebuy, :], y=["earning",'flucRate'])
            fig_stock.show()
        return data_stock



In [52]:

if __name__ == "__main__":
    start_time = time.time()
    SANA = StockAnalysis()
    daily, daily_rate = SANA.get_eco()
    industry_list_NYSE, industry_list_NASDAQ = SANA.get_overall()

 #   inds = input("Please enter your Industry preference(exp: [A, B, C]): ").strip('][').split(', ')
 #   inds = ['house', 'drug']Auto Manufacturers
    # print(inds)
    # stocks_profile, stocks_suggestion = SANA.find_industry(inds)
    print("Total time consuming --- %s seconds ---" % (time.time() - start_time))

Overall PE of NYSE: 50.771
Overall PE of NASDAQ: 52.101
Interval accepted: 41.1488 36.005199999999995


NYSE: ['Airlines' 'Auto Manufacturers' 'Banks—Diversified' 'Building Materials'
 'Computer Hardware' 'Drug Manufacturers—General'
 'Household & Personal Products' 'Oil & Gas Midstream' 'REIT—Diversified'
 'Telecom Services' 'Utilities—Regulated Electric']


NASDAQ: ['Confectioners' 'Credit Services' 'Drug Manufacturers—General'
 'Entertainment' 'Insurance—Reinsurance' 'Internet Retail' 'Leisure'
 'Oil & Gas E&P' 'Packaged Foods' 'REIT—Mortgage' 'Railroads'
 'Semiconductors' 'Steel' 'Utilities—Regulated Electric']
Total time consuming --- 96.63826370239258 seconds ---


- The **Altman Z-score** is the output of a credit-strength test that gauges a publicly traded manufacturing company's likelihood of bankruptcy.
An Altman Z-score close to 0 suggests a company might be headed for bankruptcy. The higher Altman Z-score implies the stronger and more stable business operation.

- The **Piotroski score** is a discrete score between zero and nine that reflects nine criteria used to determine the strength of a firm's financial position, evaluating the financial arrangment of a company.
The Piotroski score is used to determine the best value stocks, with nine being the best and zero being the worst.

***Revenue is the total amount of money generated by a company. Sales are the total consideration accrued from selling goods or services by a company. Sales are a subset of revenue. And sometimes, revenue can also be lower than in sales.***

Net income indicates a company's profit after all of its expenses have been deducted from revenues.\
Gross profit refers to a company's profits earned after subtracting the costs of producing and distributing its products. 

- **grossProfitRatio** = (Sales – (Direct materials + Direct Labor + Overhead)) ÷ Sales\
 `On the face of it, a gross profit margin ratio of 50 to 70% would be considered healthy, and it would be for many types of businesses, like retailers, restaurants, manufacturers and other producers of goods`\
<span style='color:Blue'> Cost of selling V.S. Income of selling.  </span>



- **ebitdaratio** = EBITDA ÷ Net Sales  \
EBITDA — Earnings(net profit or simply net income) Before Interest, Taxes, Depreciation, and Amortization\
`An EBITDA margin of 10% or more is typically considered good, as S&P-500-listed companies have EBITDA margins between 11% and 14% for the most part. You can, of course, review EBITDA statements from your competitors if they're available — be they a full EBITDA figure or an EBITDA margin percentage.`\
<span style='color:Blue'> How efficient the selling of products to earning is.  </span>


----------------------- 

- **operatingIncomeRatio** = (Gross Profit - Operating Expenses - Depreciation) ÷ Revenue \
Profit earned from a company's core business operations. It is gross profit less all operating expenses and depreciation.\
`A company that has an operating profit margin higher than 9.35% would have outperformed the overall market. However, it is essential to consider that average profit margins vary significantly between industries.`\
<span style='color:Blue'> Evaluate the internal friction of the company.  </span>


- **netIncomeRatio** = (Revenue - Cost of Goods Sold - Other Expenses) ÷ Revenue \
Net income is what remains of a company's revenue after subtracting all costs. It is also referred to as net profit, earnings, or the bottom line. Net Income that is not paid out in dividends is added to retained earnings. \
(sales minus cost of goods sold, selling, general and administrative expenses, operating expenses, depreciation, interest, taxes, and other expenses).\
`A 10% net profit margin is considered average, a 20% margin is considered high (or “good”), and a 5% margin is low.`\
<span style='color:Blue'> Evaluate how much the selling of the products contribute to the company's income .  </span>

1.  Net Profit can be more than Operating Profit. This is possible when the indirect incomes (Amounts not directly attributable to the company's main business like interest, rent etc ) are higher than the amount of indirect expenses. It is generally not normal to have an net profit more than operating ratio but it is very much possible.

Score to Recommendation:\
5 Strong buy\
4 Buy\
3 Neutral\
2 Sell\
1 Strong Sell

For names of industries, please refer to \
https://financialmodelingprep.com/api/v4/standard_industrial_classification/all?apikey=2c2cd38706ef0b12bdbf80a33806aa32



In [53]:

if __name__ == "__main__":
    start_time = time.time()
    SANA = StockAnalysis()
    inds = input("Please enter your Industry preference(exp: [A, B, C]): ").strip('][').split(', ')
    #inds = ['house', 'drug']
    print(inds)
    stocks_profile, stocks_suggestion = SANA.find_industry(inds)
    print("Total time consuming --- %s seconds ---" % (time.time() - start_time))

Please enter your Industry preference(exp: [A, B, C]): [ELECTRIC]
['ELECTRIC']
Total stocks considered:  9656
['1731', '3433', '3600', '3620', '3634', '3640', '3690', '3825', '4900', '4911', '4931', '5063', '5064']
Stocks in industry: ['ADN', 'ADNWW', 'AEE', 'AEHR', 'AEP', 'AEPPL', 'AEPPZ', 'AGQPF', 'AGR', 'AILIH', 'AILIM', 'AILIN', 'AILIO', 'AILIP', 'AILLI', 'AILLL', 'AILLM', 'AILLN', 'AILLO', 'AILLP', 'AILNP', 'ALE', 'ALP-PQ', 'ALPVN', 'AMOT', 'APRCP', 'APRDM', 'APRDN', 'APRDO', 'APRDP', 'AQN', 'AQNA', 'AQNB', 'ARTX', 'AT', 'ATKR', 'AUSI', 'AUVI', 'AVA', 'AXPWQ', 'AY', 'AYI', 'AZRE', 'AZZ', 'BE', 'BEP', 'BEPC', 'BEP-PA', 'BIIO', 'BKH', 'BLDP', 'BLXX', 'BOTY', 'BW', 'BWSN', 'BYRN', 'CAE', 'CAPC', 'CBAT', 'CCF', 'CEPU', 'CHGI', 'CIG', 'CIG-C', 'CMS', 'CMSA', 'CMSC', 'CMS-PB', 'CNP', 'CNP-PB', 'COHU', 'CPWR', 'CWEN', 'CWEN.A', 'D', 'DAIO', 'DCRB', 'DCRBU', 'DCRBW', 'DCUE', 'DRUA', 'DTE', 'DTJ', 'DTP', 'DTW', 'DTY', 'DUK', 'DUKB', 'DUKH', 'DUK-PA', 'EAF', 'EAI', 'EBR', 'EBR-B', 'ECOX', '

,industry,price,range,beta,isEtf,isFund,currentLoc,altmanZScore,piotroskiScore,reportedCurrency,grossProfitRatio,ebitdaratio,operatingIncomeRatio,netIncomeRatio,stocktwitsSentiment,twitterSentiment,sentimentChange,sentimentChangeRank
symbol,,,,,,,,,,,,,,,,,,
AEE,Utilities—Regulated Electric,94.79,79.35-94.85,0.35,False,False,1.00,0.982683,4.0,USD,0.467782,0.439005,0.208477,0.154833,NaN,NaN,NaN,NaN
AEP,Utilities—Regulated Electric,101.51,80.22-101.62,0.34,False,False,0.99,0.930642,6.0,USD,0.607664,0.407879,0.20315,0.148172,NaN,NaN,2641.6667,12.0
AEPPZ,Utilities—Regulated Electric,56.61,48.75-56.7,0.34,False,False,0.99,26.397967,7.0,USD,0.607664,0.407879,0.20315,0.148172,NaN,NaN,NaN,NaN
AGR,Utilities—Regulated Electric,47.49,42.2-55.57,0.34,False,False,0.40,0.900236,6.0,USD,0.3655,0.282191,0.128334,0.101377,NaN,NaN,NaN,NaN
AMOT,Electronic Components,29.77,28.55-44.7,1.66,False,False,0.08,2.210771,5.0,USD,0.300003,0.065299,0.064498,0.05971,NaN,NaN,NaN,NaN
APRCP,Utilities—Regulated Electric,98.87,98.87-105.0,0.00,False,False,0.00,1.309234,3.0,USD,0.479495,0.463122,0.281615,0.195384,NaN,NaN,NaN,NaN
APRDP,Utilities—Regulated Electric,100.91,96.0-104.98,0.00,False,False,0.55,0.621886,7.0,USD,0.479495,0.463122,0.281615,0.195384,NaN,NaN,NaN,NaN
ATKR,Electrical Equipment & Parts,98.39,65.802-119.96,2.38,False,False,0.60,5.639081,7.0,USD,0.384429,0.309415,0.272864,0.20077,NaN,NaN,NaN,NaN
AVA,Utilities—Diversified,45.75,37.73-48.89,0.60,False,False,0.72,0.830677,6.0,USD,0.65452,0.343105,0.158612,0.102391,NaN,NaN,NaN,NaN


Total time consuming --- 88.07850956916809 seconds ---


In [50]:
stocks_suggestion

,date,rating,ratingScore,ratingDetailsDCFScore,ratingDetailsROEScore,ratingDetailsROAScore,ratingDetailsDEScore,ratingDetailsPEScore,ratingDetailsPBScore
symbol,,,,,,,,,
AEE,2022-04-01,S-,5,5,3,3,5,5,5
AEP,2022-04-01,S-,5,5,3,3,5,5,5
AEPPZ,2022-03-25,S-,5,5,3,3,5,5,5
AGR,2022-04-01,A+,5,5,3,3,4,5,5
ALE,2022-04-01,A+,5,5,3,3,4,5,5
...,...,...,...,...,...,...,...,...,...
TVC,2022-01-31,A,4,5,3,3,5,3,3
UTL,2022-04-01,S-,5,5,3,3,5,5,5
WCC,2022-04-01,S-,5,5,3,3,5,5,5


In [58]:
if __name__ == "__main__":
    SANA = StockAnalysis()
    stocks = input("Please enter Stock symbols on your watchlist (exp: [A,B,C]): ").strip('][').split(', ')
    data_stock,data_sentiment = SANA.stock(stocks)

    # wl_stocks = input("Please enter Stock symbols on your watchlist (exp: [A,B,C]): ").strip('][').split(', ')
    # wl_dates = input("Please enter dates to buy and sellS (exp: [2021-02-25,2022-02-25]): ").strip('][').split(', ')
    # wl_money = float(input("Please enter invest money per stock(exp: 100): "))
    # mock_stock= SANA.mockowen(wl_stocks, wl_dates[0], wl_dates[1], wl_money)

Please enter Stock symbols on your watchlist (exp: [A,B,C]): [WEC, TER]
